# Install openai-agents SDK

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.5 MB/s eta 0:00:00


# Make your Jupyter Notebook capable of running asynchronous functions.

In [2]:
import nest_asyncio
nest_asyncio.apply()

# Run Google Gemini with OPENAI-Agent SDK

In [3]:
import os
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig
from google.colab import userdata

from agents import (
    Agent,
    Runner,
    set_default_openai_api,
    set_default_openai_client,
    set_tracing_disabled,
)

gemini_api_key = userdata.get("GOOGLE_API_KEY_1")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai

external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

set_default_openai_client(client=external_client, use_for_tracing=False)
set_default_openai_api("chat_completions")
set_tracing_disabled(disabled=True)

# Streaming Text code

Here’s what’s happening, line by line:

```python
result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
```

1. **`Runner.run_streamed(...)`**  
   - Instead of running the agent to completion and returning a single final string, this kicks off a *streaming* run.  
   - You pass in your `agent` instance and the user prompt (`"Please tell me 5 jokes."`).  
   - It immediately returns a `StreamedRunResult` (let’s call it `result`) which you can asynchronously iterate to receive partial outputs as they arrive.

```python
async for event in result.stream_events():
```

2. **`async for`**  
   - This is an asynchronous loop over `result.stream_events()`, which yields every event emitted during the run.  
   - Events can include things like “token received,” “tool call started,” “tool call finished,” and so on.

```python
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
```

3. **Filtering for text deltas**  
   - Each `event` has a `.type` (a simple string) and a `.data` payload (an object with more details).  
   - We check two things:  
     1. `event.type == "raw_response_event"`: this means it’s part of the model’s chat response, not a tool call or metadata.  
     2. `isinstance(event.data, ResponseTextDeltaEvent)`: among response events, we further narrow to those carrying *text delta* updates (i.e. snippets of the assistant’s reply as it’s generated).

```python
        print(event.data.delta, end="", flush=True)
```

4. **Printing out the partial text**  
   - Each `ResponseTextDeltaEvent` has a `.delta` property: a string fragment (often a word or sub-word piece) that the model just produced.  
   - By doing `print(..., end="", flush=True)`, you:  
     - Append that fragment to the console without adding a newline.  
     - `flush=True` forces Python to write it out immediately, so you see the text appear in real time as it streams in.

---

In [6]:
import asyncio

from openai.types.responses import ResponseTextDeltaEvent

from agents import Agent, Runner


async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
        model=model
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)



asyncio.run(main())

Alright, here are 5 jokes for you:

1.  Why don't scientists trust atoms?
    Because they make up everything!

2.  Parallel lines have so much in common.
    It's a shame they'll never meet.

3.  Why did the scarecrow win an award?
    Because he was outstanding in his field!

4.  What do you call a lazy kangaroo?
    Pouch potato!

5.  Why did the bicycle fall over?
    Because it was two tired!


# Stream item code
Let’s break it down piece by piece.

---

## 1. The `@function_tool`–decorated helper

```python
@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)
```

1. **`@function_tool` decorator**  
   This annotation tells your agent framework, “Please expose this Python function as a callable tool.” Under the hood, the library will wrap `how_many_jokes` so that when the agent decides it needs to call a “tool” named `how_many_jokes`, it actually invokes this function and returns its result back into the agent’s reasoning loop.

2. **Function signature**  
   - **Name**: `how_many_jokes`  
   - **Return type**: `int` (as hinted by the Python type annotation `-> int`)

3. **Function body**  
   ```python
   return random.randint(1, 10)
   ```
   - Uses Python’s standard `random` module to pick an integer between 1 and 10, inclusive.
   - Every time the agent calls this tool, it’ll get back a fresh random number in that range—useful, for example, if your agent wants to decide “How many jokes should I tell?” dynamically.

---

## 2. Advanced event‐loop processing

```python
async for event in result.stream_events():
    # We'll ignore the raw responses event deltas
    if event.type == "raw_response_event":
        continue

    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
        continue

    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print("-- Tool was called")
        elif event.item.type == "tool_call_output_item":
            print(f"-- Tool output: {event.item.output}")
        elif event.item.type == "message_output_item":
            print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
        else:
            pass  # Ignore other event types
```

Here you’re iterating over **all** of the streaming events that the agent emits, not just the text tokens. Let’s unpack each branch:

1. **`async for event in result.stream_events()`**  
   - As before, you’re listening asynchronously to a firehose of events emitted during the run.

2. **Ignoring “raw” text deltas**  
   ```python
   if event.type == "raw_response_event":
       continue
   ```
   - These are the low-level token fragments (`ResponseTextDeltaEvent`) we saw earlier. Here you’ve decided you don’t care about printing each chunk of text directly, so you skip them.

3. **Agent‐updated events**  
   ```python
   elif event.type == "agent_updated_stream_event":
       print(f"Agent updated: {event.new_agent.name}")
       continue
   ```
   - Sometimes the framework will swap in a new “agent” mid‐run (for example, if your agent calls a sub‐agent or changes strategy).  
   - When that happens, you get an `agent_updated_stream_event`, and `event.new_agent` holds the fresh agent object. You simply log its name.

4. **Run‐item events**  
   ```python
   elif event.type == "run_item_stream_event":
       ...
   ```
   - A “run item” can be a tool invocation, the tool’s result, or an ordinary chat message. You inspect `event.item.type` to distinguish:

   - **`tool_call_item`**  
     ```python
     print("-- Tool was called")
     ```
     Fired when the agent decides “I’m going to call a tool now.” No output yet, just the invocation.

   - **`tool_call_output_item`**  
     ```python
     print(f"-- Tool output: {event.item.output}")
     ```
     After the tool runs, you get this event along with `event.item.output`, the actual return value (e.g. the integer from `how_many_jokes`).

   - **`message_output_item`**  
     ```python
     print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
     ```
     When the agent emits a normal chat‐style response message (e.g. “Here are your jokes…”), it’s packaged as a `message_output_item`. You use `ItemHelpers.text_message_output(...)` to extract the full text from that item and print it.

   - **Other items**  
     There are a few other event/item types (tool‐error, metadata, etc.). You’ve chosen to ignore them with `pass`.

---

## 3. Why `asyncio.run(main())`?

At the bottom of many Python async scripts, you’ll often see:

```python
asyncio.run(main())
```

- **What it does**  
  - `main()` is your top‐level `async def` function that kicks off everything (creates the agent, runs the prompt, loops over events, etc.).  
  - `asyncio.run(...)` creates an event loop, schedules `main()` on it, runs until `main()` completes, then closes the loop cleanly.

- **Why you need it**  
  In a script (outside of an interactive REPL or notebook), you can’t just call `await main()` at the top level. You must bootstrap the async runtime yourself—and `asyncio.run` is the recommended way in Python 3.7+.

---

In [7]:
import asyncio
import random

from agents import Agent, ItemHelpers, Runner, function_tool


@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


async def main():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        model=model
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",

    )
    print("=== Run starting ===")
    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types


asyncio.run(main())

print("=== Run complete ===")

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 8
-- Message output:
 Ok, I will tell you 8 jokes.

Why don't scientists trust atoms?
Because they make up everything!

What do you call a lazy kangaroo?
Pouch potato!

What do you call a sad strawberry?
A blueberry!

Why did the bicycle fall over?
Because it was two tired!

What musical instrument is found in the bathroom?
A tuba toothpaste.

Why did the teddy bear say no to dessert?
Because she was stuffed.

Knock, knock!
Who's there?
Lettuce.
Lettuce who?
Lettuce in, it's cold out here!

Why did the scarecrow win an award?
Because he was outstanding in his field!

=== Run complete ===
